<span style="color:red">**Insert Project Token (us the menu in the top right corner). If you don't complete this step, you will get an error in write_data_by_state()**</span>

## Lineage Pipeline 2

In [ ]:
# Run once to install the Databand library
!pip install databand

In [ ]:
# Import pandas and databand libraries
import pandas as pd
from dbnd import dbnd_tracking, task, dataset_op_logger

In [ ]:
# Global variables

databand_url = 'insert_url'
databand_access_token = 'insert_token'

# Data used in this pipeline
RETAIL_FILE = "https://raw.githubusercontent.com/elenalowery/data-samples/main/Retail_Products_and_Customers.csv"

# Provide a unique suffix that will be added to various assets tracked in Databand. We use this approach because
# in a workshop many users are running the same sample pipelines. For example '_mi'
unique_suffix = '_mi'

In [ ]:
@task
def read_sales_data():
    
    url = 'https://raw.githubusercontent.com/elenalowery/data-samples/main/Camping_Equipment.csv'
    
    retailData = pd.read_csv(url)
    
    # Log the data read
    with dataset_op_logger("local://Weekly_Sales/Camping_Equipment.csv", "read", with_schema=True, with_preview=True) as logger:
        logger.set(data=retailData)
    
    return retailData

    # Unique name for logging - this matches the name used in LineagePipeline1
    unique_file_name = 'local://Weekly_Sales/Camping_Equipment.csv' + unique_suffix

    # Log the data read
    with dataset_op_logger(unique_file_name, "read", with_schema=True, with_preview=True, with_stats=True,
                           with_histograms=True, ) as logger:
        retailData = pd.read_csv(INPUT_FILE)
        logger.set(data=retailData)

    return retailData

In [ ]:
@task
def write_data_by_state(salesData):

    # Select any product line - we will write it to a separate file
    salesByState = salesData.loc[salesData['State'] == 'Arizona']
    
    # Log the filtered data read
    with dataset_op_logger("local://Weekly_Sales/Arizona_Camping_Equipment.csv", "write", with_schema=True, with_preview=True) as logger:
        logger.set(data=salesByState)
        
    project.save_data("Arizona_Camping_Equipment.csv", salesByState.to_csv(index=False), overwrite=True)

In [ ]:
# Call and track all steps in a pipeline

# TODO: 
# - Update databand URL and token
# - Update project and job name (add your initials)

def prepare_retail_data():

    with dbnd_tracking(
            conf={
                "core": {
                    "databand_url": databand_url,
                    "databand_access_token": databand_access_token,
                }
            },
            job_name="lineage_pipeline_2_notebook",
            run_name="weekly",
            project_name="Lineage Test",
    ):

        #Call the step job - read data
        salesData = read_sales_data()

        # Write data by product line
        write_data_by_state(salesData)

        print("Finished running the pipeline")

# Invoke the main function
prepare_retail_data()